In [ ]:
import os
import shutil
import random
import itertools
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib as mpl
from keras import backend
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications import imagenet_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenetv2 import decode_predictions, preprocess_input

In [ ]:
# Load data

labels = ['1', '0']
train_path = '.../Data/Dataset/foder_origin/train'
valid_path = '.../Data/Dataset/foder_origin/validation'

# Creating batches of data
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)


In [ ]:
# Load pre-trained mobilenet model
from tensorflow.keras.applications import MobileNetV2

mobile = MobileNetV2(weights='imagenet', include_top=False)
mobile.summary()

In [ ]:
# Only training the last 12 layers of the mobilenet during finetuning as we want
# Keep all of the previously learned weights
x = mobile.layers[-12].output
x

In [ ]:
# Add pooling, dropout layer
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
output = Dense(units=2, activation='sigmoid')(x)

In [ ]:
# Hoàn thiện model với output tùy chỉnh
model = Model(inputs=mobile.input, outputs=output)

In [ ]:
# Keep the weights the same except for the last 23 layers of the model ==> Retain the features from the original MobileNet model
for layer in model.layers[:-23]:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Using Early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(x=train_batches,
          steps_per_epoch=len(train_batches),
          validation_data=valid_batches,
          validation_steps=len(valid_batches),
          epochs=20,
          verbose=2,
          callbacks=[early_stopping]
)
history

In [ ]:
# Saving model

model.save("/gdrive/MyDrive//Lab1_DPL302m/modelsaving")
#history = model.load_weights('/gdrive/MyDrive//Lab1_DPL302m/modelsaving')

In [ ]:
# Evaluating
from sklearn.metrics import confusion_matrix, precision_score, f1_score as sklearn_f1_score, accuracy_score

test_labels = test_batches.classes
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))
precision = precision_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
f1_score = sklearn_f1_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
accuracy = accuracy_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))

# Print precision, F1 score and accuracy of our model
print('Precision: ', precision)
print('F1 Score: ', f1_score)
print('Accuracy: ', accuracy)



In [ ]:
import matplotlib.pyplot as plt

# Draw plot Training và Validation Accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Draw plot Training và Validation Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:

# Write predicted values ​​to CSV file

import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image


labels = ['1', '0']

def preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

def predict_image(file_path):
    preprocessed_image = preprocess_image(file_path)
    predictions = model.predict(preprocessed_image)
    result = np.argmax(predictions)
    return labels[result]


dataset_dir = '.../Data/Dataset/testset'

output_csv = '.../Data/Dataset/Predict_test.csv'

predictions_list = []


for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        file_path = os.path.join(dataset_dir, filename)
        prediction = predict_image(file_path)
        filename_without_extension = os.path.splitext(filename)[0]
        predictions_list.append((filename_without_extension, prediction))


with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'Prediction'])
    for prediction in predictions_list:
        writer.writerow(prediction)

print("Predictions saved to", output_csv)